In [ ]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import xgboost as xgb
from google.colab import files

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
uploaded = files.upload()

Saving sentiment_analysis.csv to sentiment_analysis.csv


In [ ]:
df = pd.read_csv('sentiment_analysis.csv')
df = df.drop(['ID'], axis=1)
df.head()

,text,label
0,Josh Jenkins is looking forward to TAB Breeder...,1
1,RT @MianUsmanJaved: Congratulations Pakistan o...,1
2,"RT @PEPalerts: This September, @YESmag is taki...",1
3,"RT @david_gaibis: Newly painted walls, thanks ...",1
4,RT @CedricFeschotte: Excited to announce: as o...,1


In [ ]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)

    #removes RT
    _sentence = re.sub("RT",'',_sentence)

    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))

    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df['text'] = vfunc_clean_text(tweets_df['text']) #create a new column that consists of preprocessed tweets

    return tweets_df

In [ ]:
df = preprocess_tweets(df)

In [ ]:
df

,text,label
0,josh jenkins looking forward tab breeders crow...,1
1,congratulations pakistan becoming world odds,1
2,september taking maine mendoza’s surprise than...,1
3,newly painted walls thanks million custodial p...,1
4,excited announce july 2017 feschotte lab reloc...,1
...,...,...
550386,cant stop watching hermm,0
550387,poor old tom odell doesnt look like would know...,0
550388,smashed 7 ants awesome gamehjfjfi,1
550389,morning girls wonderful,1


In [ ]:
X = df.loc[:, ['text']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
vectorizer = CountVectorizer(binary=False,ngram_range=(1,1))
X_train_ = vectorizer.fit_transform(X_train['text'])
X_test_ = vectorizer.transform(X_test['text'])
print (X_train.shape, X_test.shape)

(440312, 1) (110079, 1)


TV = TfidfVectorizer(max_features = 1000)   
X_train = TV.fit_transform(X_train['text'])
X_test = TV.fit_transform(X_test['text'])
print(np.shape(X_train))
print(np.shape(X_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def model_training(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    acc_train = metrics.accuracy_score(y_train, y_pred_train)
    acc_test = metrics.accuracy_score(y_test, y_pred_test)

    #Print Accuracies for comparison
    print("The accuracy score for the Train set is: {}".format(acc_train))
    print("The accuracy score for the Test set is: {}".format(acc_test))
    print(classification_report(y_test, y_pred_test))
    print(confusion_matrix(y_test, y_pred_test, labels=[0, 1]))



In [ ]:
KNN_model = KNeighborsClassifier()
print("Results when KNN model is used:\n")
model_training(KNN_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
LR_model = LogisticRegression()
print("Results when Logistic Regression is used:\n")
model_training(LR_model, X_train_, y_train,  X_test_, y_test)

Results when Logistic Regression is used:

The accuracy score for the Train set is: 0.9834617271389379
The accuracy score for the Test set is: 0.973310077308115
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     35810
           1       0.98      0.98      0.98     74269

    accuracy                           0.97    110079
   macro avg       0.97      0.97      0.97    110079
weighted avg       0.97      0.97      0.97    110079

[[34457  1353]
 [ 1585 72684]]


In [ ]:
SVC_model = LinearSVC()
print("Results when SVC model is used:\n")
model_training(SVC_model, X_train_, y_train,  X_test_, y_test)

Results when SVC model is used:

The accuracy score for the Train set is: 0.9910472573992987
The accuracy score for the Test set is: 0.971429609643983
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     35810
           1       0.98      0.98      0.98     74269

    accuracy                           0.97    110079
   macro avg       0.97      0.97      0.97    110079
weighted avg       0.97      0.97      0.97    110079

[[34348  1462]
 [ 1683 72586]]


In [ ]:
MultinomialNB_model = MultinomialNB()
print("Results when Naive Bayes model is used:\n")
model_training(MultinomialNB_model, X_train_, y_train,  X_test_, y_test)

Results when Naive Bayes model is used:

The accuracy score for the Train set is: 0.964629626265012
The accuracy score for the Test set is: 0.9534243588695391
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     35810
           1       0.96      0.97      0.97     74269

    accuracy                           0.95    110079
   macro avg       0.95      0.94      0.95    110079
weighted avg       0.95      0.95      0.95    110079

[[32681  3129]
 [ 1998 72271]]


In [ ]:
randomforest_model = RandomForestClassifier()
print("Results when Random Forest model is used:\n")
model_training(randomforest_model, X_train_, y_train,  X_test_, y_test)

Results when Random Forest model is used:



In [ ]:
decisiontree_model = DecisionTreeClassifier()
print("Results when Decision Tree model is used:\n")
model_training(decisiontree_model, X_train_, y_train,  X_test_, y_test)

Random forest classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from scipy.stats import randint

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
vectorizer = CountVectorizer(binary=False,ngram_range=(1,1))
X_train1_ = vectorizer.fit_transform(X_train1['text'])
X_test1_ = vectorizer.transform(X_test1['text'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier()
scores = cross_val_score(rf,X_train1_,y_train1.values.ravel(),cv=5)

In [ ]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

cv = GridSearchCV(rf,parameters)
cv.fit(X_train1_,y_train1.values.ravel())
cv.best_estimator_

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Convert text column to list
X_train1_text = X_train1['text'].tolist()
X_test1_text = X_test1['text'].tolist()

# Fit CountVectorizer and transform text data
vectorizer = CountVectorizer(binary=False, ngram_range=(1, 1))
X_train1_ = vectorizer.fit_transform(X_train1_text)
X_test1_ = vectorizer.transform(X_test1_text)

# Create DMatrix object with categorical feature enabled
dtrain = xgb.DMatrix(data=X_train1_, label=y_train1, enable_categorical=True)
dtest = xgb.DMatrix(data=X_test1_, enable_categorical=True)

# Train XGBoost model
params = {'max_depth': 3, 'eta': 0.1, 'objective': 'binary:logistic'}
xgb_model = xgb.train(params=params, dtrain=dtrain)

# Make predictions
y_predictions_train = xgb_model.predict(dtrain)
y_predictions_test = xgb_model.predict(dtest)

# Convert predicted probabilities to binary values using a threshold of 0.5
y_pred_train = (y_predictions_train > 0.5).astype(int)
y_pred_test = (y_predictions_test > 0.5).astype(int)

# Calculate accuracies
acc_train = metrics.accuracy_score(y_train1, y_pred_train, normalize=True, sample_weight=None)
acc_test = metrics.accuracy_score(y_test1, y_pred_test, normalize=True, sample_weight=None)

# Print accuracies and other metrics
print("The accuracy score for the Train set is: {}".format(acc_train))
print("The accuracy score for the Test set is: {}".format(acc_test))
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test, labels=[0, 1]))



The accuracy score for the Train set is: 0.6749009793055833
The accuracy score for the Test set is: 0.674951625650669
              precision    recall  f1-score   support

           0       0.33      0.00      0.00     35810
           1       0.67      1.00      0.81     74269

    accuracy                           0.67    110079
   macro avg       0.50      0.50      0.40    110079
weighted avg       0.56      0.67      0.54    110079

[[   17 35793]
 [   34 74235]]
